# Streaming Platforms Analysis

## SQL Queries

### Database Schema

Created database, schema, and 2 tables:

    CREATE DATABASE IF NOT EXISTS STREAMING_DATA;

    USE DATABASE STREAMING_DATA;
    CREATE SCHEMA IF NOT EXISTS CONTENT_ANALYSIS;

    USE SCHEMA STREAMING_DATA.CONTENT_ANALYSIS;
    CREATE TABLE IF NOT EXISTS titles (
      id STRING,
      title STRING,
      type STRING,
      description STRING,
      release_year INT,
      age_certification STRING,
      runtime INT,
      genres STRING,
      production_countries STRING,
      seasons INT,
      imdb_id STRING,
      imdb_score FLOAT,
      imdb_votes INT,
      platform STRING
    );

    CREATE TABLE IF NOT EXISTS credits (
      person_id INT,
      id STRING,
      name STRING,
      character_name STRING,
      role STRING,
      platform STRING
    );

Load data with the following parameters:
<br>File format: Delimited Files (CSV or TSV)
<br>Header: Skip first line
<br>Field optionally enclosed by: Double quotes
<br>All other parameters by default

### In general (across all platforms)

1) **Top rated titles:** the highest-rated movies or shows based on IMDb scores (maybe separately for shows and movies)

--- top 10 movies across all platforms with statistically significant number of votes (50 and above):

    select id, title, type, release_year, runtime, genres, production_countries, imdb_score, imdb_votes, platform
    from titles
    where type = 'MOVIE' and imdb_votes >= 50 
    order by imdb_score desc
    limit 10;

 --- top 10 tv shows across all platforms with statistically significant number of votes (50 and above):
 
    select id, title, type, release_year, runtime, genres, production_countries, seasons, imdb_score, imdb_votes, platform
    from titles
    where type = 'SHOW' and imdb_votes >= 50 
    order by imdb_score desc
    limit 10;

2) **Most popular titles:** titles with the highest IMDb number of votes (also separately for shows and movies) - viewer engagement.

 --- top 10 movies across all platforms with the highest IMDb number of votes:
 
    select id, title, type, release_year, runtime, genres, production_countries, imdb_score, imdb_votes, platform
    from titles
    where type = 'MOVIE'
    order by imdb_votes desc
    limit 10;

--- top 10 tv shows across all platforms with the highest IMDb number of votes:

    select id, title, type, release_year, runtime, genres, production_countries, seasons, imdb_score, imdb_votes, platform
    from titles
    where type = 'SHOW'
    order by imdb_votes desc
    limit 10;

3) **Popular genres:** the most popular genres based on the number of titles and/or average ratings (might be challenging since many movies have several genres).

--- number of movies and tv shows by genres by platforms

    select platform, type, trim(genre.value, ' []') as genres, count(distinct titles.id) as content_qty
    from titles, lateral split_to_table(titles.genres, ',') as genre
    group by 1, 2, 3
    order by 1, 4 desc;
 
 ---number of movies by genres across all platforms
 
    select trim(genre.value, ' []') as genres, count(distinct titles.id) as movies_qty --  used distinct because there are some movies represented on several platforms
    from titles, lateral split_to_table(titles.genres, ',') as genre
    where titles.type = 'MOVIE' -- there are 155 movies without genre
    group by 1
    order by 2 desc; 
---number of shows by genres across all platforms

    select trim(genre.value, ' []') as genres, count(distinct titles.id) as shows_qty -- used distinct because there are some shows represented on several platforms
    from titles, lateral split_to_table(titles.genres, ',') as genre
    where titles.type = 'SHOW' -- there are 119 shows without genre
    group by 1
    order by 2 desc;

---average rating of movie genres across all platforms

    with t as
    (select distinct id, title, type, genres, imdb_score -- used distinct because there are some movies represented on several platforms
    from titles
    where titles.type = 'MOVIE')
    select trim(genre.value, ' []') as genres, avg(t.imdb_score) as avg_movies_imdb_score
    from t, lateral split_to_table(t.genres, ',') as genre
    group by 1
    order by 2 desc;

---average rating of show genres across all platforms

    with t as
    (select distinct id, title, type, genres, imdb_score --used distinct because there are some shows represented on several platforms
    from titles
    where titles.type = 'SHOW')
    select trim(genre.value, ' []') as genres, avg(t.imdb_score) as avg_shows_imdb_score
    from t, lateral split_to_table(t.genres, ',') as genre
    group by 1
    order by 2 desc;

4) **Genres trends over time:** how the popularity of genres has changed over the years.

 ---number of movies released by years and genres across all platforms
 
    select release_year, trim(genre.value, ' []') as genres, count(distinct id) as movies_qty
    from titles, lateral split_to_table(titles.genres, ',') as genre
    where type = 'MOVIE'
    group by 1, 2
    order by 1 desc, 3 desc;

---number of shows released by years and genres across all platforms

    select release_year, trim(genre.value, ' []') as genres, count(distinct id) as shows_qty
    from titles, lateral split_to_table(titles.genres, ',') as genre
    where type = 'SHOW'
    group by 1, 2
    order by 1 desc, 3 desc;

5) **Release year trends:** trends in the release years of shows and movies to see how content production has evolved.

 ---number of movies released by years across all platforms
 
    select release_year, count(distinct id) as movies_qty
    from titles
    where type = 'MOVIE'
    group by 1
    order by 2 desc;

---number of shows released by years across all platforms

    select release_year, count(distinct id) as shows_qty
    from titles
    where type = 'SHOW'
    group by 1
    order by 2 desc; 

6) **Target audience:** which age groups are being targeted most by these platforms based on the age certification.

---number of movies by age certification across all platforms

    select age_certification, count(distinct id) as movies_qty
    from titles
    where type = 'MOVIE'
    group by 1
    order by 2 desc; --- 9604 movies are unrated

---number of shows by age certification across all platforms

    select age_certification, count(distinct id) as shows_qty
    from titles
    where type = 'SHOW'
    group by 1
    order by 2 desc; --- 1397 shows are unrated

7) **Production country analysis:** which countries are most prolific in producing content available on these platforms.

---number of movies by country across all platforms

    select trim(country.value, ' []') as production_country, count(distinct titles.id) as movies_qty --I used distinct because there are some movies represented on several platforms
    from titles, lateral split_to_table(titles.production_countries, ',') as country
    where titles.type = 'MOVIE' -- there are 265 movies without country
    group by 1
    order by 2 desc;

---number of shows by country across all platforms

    select trim(country.value, ' []') as production_country, count(distinct titles.id) as shows_qty --I used distinct because there are some shows represented on several platforms
    from titles, lateral split_to_table(titles.production_countries, ',') as country
    where titles.type = 'SHOW' -- there are 28 shows without country
    group by 1
    order by 2 desc;

8) **Most frequent collaborations:** actors who frequently work with the same directors.

---10 most frequent actor-director movies collaborations across all platforms

    with actors as
    (select id, name as actor_name
    from credits
    where role = 'ACTOR' and contains(id, 'tm')), --- filter by movies
    directors as
    (select id, name as director_name
    from credits
    where role = 'DIRECTOR' and contains(id, 'tm')), --- filter by movies
    pairs as
    (select a.id, a.actor_name, d.director_name, concat(a.actor_name, ' with ', d.director_name) as actor_director_pair
    from actors a
    join directors d on a.id=d.id)
    select actor_director_pair, count(id) as frequency
    from pairs
    group by 1
    order by 2 desc
    limit 10;

---10 most frequent actor-director tv shows collaborations across all platforms

    with actors as
    (select id, name as actor_name
    from credits
    where role = 'ACTOR' and contains(id, 'ts')), --- filter by tv shows
    directors as
    (select id, name as director_name
    from credits
    where role = 'DIRECTOR' and contains(id, 'ts')), --- filter by tv shows
    pairs as
    (select a.id, a.actor_name, d.director_name, concat(a.actor_name, ' with ', d.director_name) as actor_director_pair
    from actors a
    join directors d on a.id=d.id)
    select actor_director_pair, count(id) as frequency
    from pairs
    group by 1
    order by 2 desc
    limit 10;

9) **Most prolific actors/directors:** which actors or directors are most prolific across these platforms.
 Actor-director impact: if certain actors or directors consistently contribute to higher-rated titles.

---actors with highest average ratings of movies they were filmed in across all platforms, and number of movies

    select c.person_id as actor_id, c.name, avg(t.imdb_score) as avg_movie_imdb_score, count(distinct t.id) as movies_qty
    from credits c
    join titles t on t.id = c.id
    where c.role = 'ACTOR' and t.type = 'MOVIE' and imdb_votes >= 50
    group by 1, 2
    order by 3 desc, 4 desc;

---directors with highest average ratings of movies they filmed across all platforms, and number of movies

    select c.person_id as director_id, c.name, avg(t.imdb_score) as avg_movie_imdb_score, count(distinct t.id) as movies_qty
    from credits c
    join titles t on t.id = c.id
    where c.role = 'DIRECTOR' and t.type = 'MOVIE' and imdb_votes >= 50
    group by 1, 2
    order by 3 desc, 4 desc;

---actors with highest average ratings of shows they were filmed in across all platforms, and number of shows

    select c.person_id as actor_id, c.name, avg(t.imdb_score) as avg_show_imdb_score, count(distinct t.id) as show_qty
    from credits c
    join titles t on t.id = c.id
    where c.role = 'ACTOR' and t.type = 'SHOW' and imdb_votes >= 50
    group by 1, 2
    order by 3 desc, 4 desc;

---directors with highest average ratings of shows they filmed across all platforms, and number of shows

    select c.person_id as director_id, c.name, avg(t.imdb_score) as avg_show_imdb_score, count(distinct t.id) as show_qty
    from credits c
    join titles t on t.id = c.id
    where c.role = 'DIRECTOR' and t.type = 'SHOW' and imdb_votes >= 50
    group by 1, 2
    order by 3 desc, 4 desc;

### Comparison between platforms: (Cross platform comparison)

10) **Average ratings:** the average IMDb scores across platforms.

---average imdb rating for movies by platforms

    select platform, avg(imdb_score) as avg_movie_imdb_score
    from titles
    where type = 'MOVIE'
    group by 1
    order by 2 desc;
 
---average imdb rating for shows by platforms

    select platform, avg(imdb_score) as avg_shows_imdb_score
    from titles
    where type = 'SHOW'
    group by 1
    order by 2 desc;

11) **Top-rated titles by platform:** the highest-rated titles on each platform (separately for shows and movies).

--- top 10 movies on Amazon with statistically significant number of votes (50 and above):

    select id, title, type, release_year, runtime, genres, production_countries, imdb_score, imdb_votes, platform
    from titles
    where type = 'MOVIE' and imdb_votes >= 50 and platform = 'amazon'
    order by imdb_score desc
    limit 10;

--- top 10 movies on Netflix with statistically significant number of votes (50 and above):

    select id, title, type, release_year, runtime, genres, production_countries, imdb_score, imdb_votes, platform
    from titles
    where type = 'MOVIE' and imdb_votes >= 50 and platform = 'netflix'
    order by imdb_score desc
    limit 10;

--- top 10 movies on Hulu with statistically significant number of votes (50 and above):

    select id, title, type, release_year, runtime, genres, production_countries, imdb_score, imdb_votes, platform
    from titles
    where type = 'MOVIE' and imdb_votes >= 50 and platform = 'hulu'
    order by imdb_score desc
    limit 10;

--- top 10 movies on Disney with statistically significant number of votes (50 and above):

    select id, title, type, release_year, runtime, genres, production_countries, imdb_score, imdb_votes, platform
    from titles
    where type = 'MOVIE' and imdb_votes >= 50 and platform = 'disney'
    order by imdb_score desc
    limit 10;

 --- top 10 shows on Amazon with statistically significant number of votes (50 and above):

    select id, title, type, release_year, runtime, genres, production_countries, seasons, imdb_score, imdb_votes, platform
    from titles
    where type = 'SHOW' and imdb_votes >= 50 and platform = 'amazon'
    order by imdb_score desc
    limit 10;

--- top 10 shows on Netflix with statistically significant number of votes (50 and above):

    select id, title, type, release_year, runtime, genres, production_countries, seasons, imdb_score, imdb_votes, platform
    from titles
    where type = 'SHOW' and imdb_votes >= 50 and platform = 'netflix'
    order by imdb_score desc
    limit 10;

--- top 10 shows on Hulu with statistically significant number of votes (50 and above):

    select id, title, type, release_year, runtime, genres, production_countries, seasons, imdb_score, imdb_votes, platform
    from titles
    where type = 'SHOW' and imdb_votes >= 50 and platform = 'hulu'
    order by imdb_score desc
    limit 10;

--- top 10 shows on Disney with statistically significant number of votes (50 and above):

    select id, title, type, release_year, runtime, genres, production_countries, seasons, imdb_score, imdb_votes, platform
    from titles
    where type = 'SHOW' and imdb_votes >= 50 and platform = 'disney'
    order by imdb_score desc
    limit 10;

12) **Most popular titles by platform:** which titles are most popular on each platform based on the number of IMDb votes.

 --- top 10 movies on Amazon with the highest IMDb number of votes:

    select id, title, type, release_year, runtime, genres, production_countries, imdb_score, imdb_votes, platform
    from titles
    where type = 'MOVIE' and platform = 'amazon'
    order by imdb_votes desc
    limit 10;

--- top 10 movies on Netflix with the highest IMDb number of votes:

    select id, title, type, release_year, runtime, genres, production_countries, imdb_score, imdb_votes, platform
    from titles
    where type = 'MOVIE' and platform = 'netflix'
    order by imdb_votes desc
    limit 10;

--- top 10 movies on Hulu with the highest IMDb number of votes:

    select id, title, type, release_year, runtime, genres, production_countries, imdb_score, imdb_votes, platform
    from titles
    where type = 'MOVIE' and platform = 'hulu'
    order by imdb_votes desc
    limit 10;

--- top 10 movies on Disney with the highest IMDb number of votes:

    select id, title, type, release_year, runtime, genres, production_countries, imdb_score, imdb_votes, platform
    from titles
    where type = 'MOVIE' and platform = 'disney'
    order by imdb_votes desc
    limit 10;

--- top 10 shows on Amazon with the highest IMDb number of votes:

    select id, title, type, release_year, runtime, genres, production_countries, imdb_score, imdb_votes, platform
    from titles
    where type = 'SHOW' and platform = 'amazon'
    order by imdb_votes desc
    limit 10;

--- top 10 shows on Netflix with the highest IMDb number of votes:

    select id, title, type, release_year, runtime, genres, production_countries, imdb_score, imdb_votes, platform
    from titles
    where type = 'SHOW' and platform = 'netflix'
    order by imdb_votes desc
    limit 10;

--- top 10 shows on Hulu with the highest IMDb number of votes:

    select id, title, type, release_year, runtime, genres, production_countries, imdb_score, imdb_votes, platform
    from titles
    where type = 'SHOW' and platform = 'hulu'
    order by imdb_votes desc
    limit 10;

--- top 10 shows on Disney with the highest IMDb number of votes:

    select id, title, type, release_year, runtime, genres, production_countries, imdb_score, imdb_votes, platform
    from titles
    where type = 'SHOW' and platform = 'disney'
    order by imdb_votes desc
    limit 10;

13) **Viewer engagement:** the level of viewer engagement (measured by the total number of IMDb votes) across platforms.

--- total number of imdb votes for movies by platforms:

    select platform, sum(imdb_votes) as total_movies_imdb_votes
    from titles
    where type = 'MOVIE'
    group by 1
    order by 2 desc;

--- total number of imdb votes for shows by platforms:

    select platform, sum(imdb_votes) as total_shows_imdb_votes
    from titles
    where type = 'SHOW'
    group by 1
    order by 2 desc;

14) **Age certification distribution:** the distribution of age certifications across platforms to understand their target demographics. 

  ---number of movies by age certification by each platform

    select platform, age_certification, count(id) as movies_qty
    from titles
    where type = 'MOVIE'
    group by 1, 2
    order by 1, 3 desc;

---number of shows by age certification by each platform

    select platform, age_certification, count(id) as shows_qty
    from titles
    where type = 'SHOW'
    group by 1, 2
    order by 1, 3 desc;

15) **Content production trends:** number of movies/shows released at each platform over years.

---number of movies released by years by each platform

    select platform, release_year, count(id) as movies_qty
    from titles
    where type = 'MOVIE'
    group by 1, 2
    order by 1, 3 desc;

---number of shows released by years by each platform

    select platform, release_year, count(id) as shows_qty
    from titles
    where type = 'SHOW'
    group by 1, 2
    order by 1, 3 desc; 

16) **Cross-platform presence of actors/directors:** which actors and directors are most common across multiple platforms.

--top 10 actors who have appeared in the most movies that are present on all four platforms

    with t as
    (select person_id, name as actor_name, count(distinct platform) as platform_presence_qty
    from credits
    where role = 'ACTOR' and contains(id, 'tm') --movies
    group by 1, 2)
    select c.person_id, c.name as actor_name, count(distinct c.id) as movies_qty
    from credits c
    join t on t.person_id=c.person_id
    where c.role = 'ACTOR' and t.platform_presence_qty = 4 and contains(c.id, 'tm') --movies
    group by 1, 2
    order by 3 desc
    limit 10;

-- top 10 actors who have appeared in the most shows that are present on all four platforms

    with t as
    (select person_id, name as actor_name, count(distinct platform) as platform_presence_qty
    from credits
    where role = 'ACTOR' and contains(id, 'ts') --shows
    group by 1, 2)
    select c.person_id, c.name as actor_name, count(distinct c.id) as shows_qty
    from credits c
    join t on t.person_id=c.person_id
    where c.role = 'ACTOR' and t.platform_presence_qty = 4 and contains(c.id, 'ts') --shows
    group by 1, 2
    order by 3 desc
    limit 10;

-- top 10 directors of the most movies that are present on all four platforms

    with t as
    (select person_id, name as director_name, count(distinct platform) as platform_presence_qty
    from credits
    where role = 'DIRECTOR' and contains(id, 'tm') --movies
    group by 1, 2)
    select c.person_id, c.name as director_name, count(distinct c.id) as movies_qty
    from credits c
    join t on t.person_id=c.person_id
    where c.role = 'DIRECTOR' and t.platform_presence_qty = 4 and contains(c.id, 'tm') --movies
    group by 1, 2
    order by 3 desc
    limit 10;

-- top 10 directors of the most shows that are present on at least 3 platforms

    with t as
    (select person_id, name as director_name, count(distinct platform) as platform_presence_qty
    from credits
    where role = 'DIRECTOR' and contains(id, 'ts') --shows
    group by 1, 2)
    select c.person_id, c.name as director_name, count(distinct c.id) as shows_qty
    from credits c
    join t on t.person_id=c.person_id
    where c.role = 'DIRECTOR' and t.platform_presence_qty >= 3 and contains(c.id, 'ts') --shows (there are only 5 directors whose shows presented on 3 platforms and none on 4 platforms)
    group by 1, 2
    order by 3 desc
    limit 10;

17) **Global reach:** the extent of international content available on each platform; if any platform has a specific focus on content from certain regions or countries.

---number of movies by country by platform

    select platform, trim(country.value, ' []') as production_country, count(titles.id) as movies_qty
    from titles, lateral split_to_table(titles.production_countries, ',') as country
    where titles.type = 'MOVIE'
    group by 1,2
    order by 1, 3 desc;

---number of shows by country by platform

    select platform, trim(country.value, ' []') as production_country, count(titles.id) as shows_qty
    from titles, lateral split_to_table(titles.production_countries, ',') as country
    where titles.type = 'SHOW'
    group by 1,2
    order by 1, 3 desc;

18) **What is the oldest movie / tv show on each platform?**

-- the oldest movie on each platform

    with t as
    (select platform, id, title, release_year, age_certification, runtime, genres, production_countries, imdb_score, imdb_votes, rank() over(partition by platform order by release_year) as ranking
    from titles
    where type = 'MOVIE')
    select * exclude ranking
    from t
    where ranking=1
    order by release_year;

-- the oldest tv show on each platform

    with t as
    (select platform, id, title, release_year, age_certification, runtime, genres, production_countries, seasons, imdb_score, imdb_votes, rank() over(partition by platform order by release_year) as ranking
    from titles
    where type = 'SHOW')
    select * exclude ranking
    from t
    where ranking=1
    order by release_year;

19) **What show has the most seasons?**

-- tv show with the most seasons on each platform

    with t as
    (select platform, id, title, release_year, age_certification, runtime, genres, production_countries, seasons, imdb_score, imdb_votes, rank() over(partition by platform order by seasons desc) as ranking
    from titles
    where type = 'SHOW')
    select * exclude ranking
    from t
    where ranking=1
    order by seasons desc;